In [ ]:
# pip install langchain

In [ ]:
import DBObject
import sqlite3
import generatePrompt
from typing import List

In [ ]:
def connect_fun(database_name: str) -> DBObject:
  conn = sqlite3.connect(f"file:{database_name}?mode=ro", uri=True) # open in readonly mode
  return DBObject(conn)



In [ ]:

def query_fun(question: str, tables: List[str], database_connection: DBObject) -> str:
  prompt = generatePrompt()

  pass